In [4]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms

transform = transforms.ToTensor()

In [5]:
mbti_df = pd.read_csv('crop128/all.csv')

photo_df = pd.DataFrame(columns=['photo'], dtype= 'object')

# put the photo data into a dataframe
for i in range(len(mbti_df)):
    image = transform(Image.open('crop128/' + mbti_df['file_name'][i]))
    photo_df.loc[i] = [image]

mbti_df = mbti_df['MBTI']

mbti_df.head()

0    enfj
1    enfj
2    enfj
3    enfj
4    enfj
Name: MBTI, dtype: object

In [6]:
photo_df.iloc[0][0].shape

torch.Size([1, 128, 128])

In [7]:
alphabet = ['t', 'f']

# if mbti_df['MBTI'] includes alphabet[0], then mbti_df['MBTI'] = 1, else 0
mbti_df = mbti_df.apply(lambda x: 1 if alphabet[0] in x else 0)

mbti_df.head()

0    0
1    0
2    0
3    0
4    0
Name: MBTI, dtype: int64

In [8]:
#print the statistics of the mbti_df
print(mbti_df.value_counts())

1    1315
0    1233
Name: MBTI, dtype: int64


In [9]:
# train_data includes the element of 2-dim tensor
train_data = photo_df['photo'].values
train_label = mbti_df.values

In [10]:
#전체 data 중 train의 비율
train_ratio = 0.8

train_idx = np.random.choice(len(train_data), int(len(train_data) * train_ratio), replace=False)
test_idx = np.array(list(set(range(len(train_data))) - set(train_idx)))

test_data = train_data[test_idx]
test_label = train_label[test_idx]

train_data = train_data[train_idx]
train_label = train_label[train_idx]


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
import keras
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

score_acc = make_scorer(accuracy_score)

In [12]:
# Create optimizer for cnn image classification
def nn_cl_bo(neurons, activation, optimizer, learning_rate,  batch_size, epochs ):
    # Create model
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(1, 32, 32)))
    model.add(keras.layers.Dense(neurons))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    # Fit the model
    model.fit(train_data, train_label, batch_size=batch_size, epochs=epochs, verbose=0)
    # Evaluate the model
    scores = model.evaluate(test_data, test_label, verbose=0)
    return scores[1]

In [13]:
from bayes_opt import BayesianOptimization

In [14]:
# Set paramaters
params_nn ={
    'neurons': (10, 100),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=123)
nn_bo.maximize(init_points=5, n_iter=10)

print(nn_bo.max)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------


TypeError: 'numpy.float64' object is not callable